# Final Project: End-to-End Data Cleaning Workflow

### Farmers Market Dataset

In [1]:
from IPython.display import display
import pandas as pd
import numpy as np
import re
import calendar
import datetime

In [2]:
df = pd.read_csv('farmers.csv')

### Raw Data 

In [3]:
with pd.option_context('display.max_columns', None):
    display(df.head(10))

,FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,street,city,County,State,zip,Season1Date,Season1Time,Season2Date,Season2Time,Season3Date,Season3Time,Season4Date,Season4Time,x,y,Location,Credit,WIC,WICcash,SFMNP,SNAP,Organic,Bakedgoods,Cheese,Crafts,Flowers,Eggs,Seafood,Herbs,Vegetables,Honey,Jams,Maple,Meat,Nursery,Nuts,Plants,Poultry,Prepared,Soap,Trees,Wine,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime
0,1018261,Caledonia Farmers Market Association - Danville,https://sites.google.com/site/caledoniafarmers...,https://www.facebook.com/Danville.VT.Farmers.M...,NaN,NaN,NaN,NaN,Danville,Caledonia,Vermont,05828,06/14/2017 to 08/30/2017,Wed: 9:00 AM-1:00 PM;,09/06/2017 to 10/18/2017,Wed: 2:00 PM-6:00 PM;,NaN,NaN,NaN,NaN,-72.140337,44.411036,NaN,Y,Y,N,Y,N,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,Y,Y,Y,N,N,N,Y,Y,Y,Y,N,Y,Y,Y,N,N,Y,Y,N,N,6/20/2017 10:43:57 PM
1,1018318,Stearns Homestead Farmers' Market,http://www.StearnsHomestead.com,StearnsHomesteadFarmersMarket,NaN,NaN,NaN,6975 Ridge Road,Parma,Cuyahoga,Ohio,NaN,06/24/2017 to 09/30/2017,Sat: 9:00 AM-1:00 PM;,NaN,NaN,NaN,NaN,NaN,NaN,-81.733939,41.374801,NaN,Y,N,N,Y,N,-,Y,N,Y,Y,Y,N,Y,Y,Y,Y,Y,N,N,N,N,Y,N,Y,N,N,N,N,Y,N,N,N,N,N,N,6/21/2017 5:15:01 PM
2,1009364,106 S. Main Street Farmers Market,http://thetownofsixmile.wordpress.com/,NaN,NaN,NaN,NaN,106 S. Main Street,Six Mile,NaN,South Carolina,29682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-82.818700,34.804200,NaN,Y,N,N,N,N,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013
3,1010691,10th Steet Community Farmers Market,NaN,NaN,NaN,NaN,http://agrimissouri.com/mo-grown/grodetail.php...,10th Street and Poplar,Lamar,Barton,Missouri,64759,04/02/2014 to 11/30/2014,Wed: 3:00 PM-6:00 PM;Sat: 8:00 AM-1:00 PM;,NaN,NaN,NaN,NaN,NaN,NaN,-94.274619,37.495628,NaN,Y,N,N,N,N,-,Y,N,Y,N,Y,N,Y,Y,Y,Y,N,Y,N,N,Y,Y,Y,Y,N,N,N,N,Y,N,N,N,N,N,N,10/28/2014 9:49:46 AM
4,1002454,112st Madison Avenue,NaN,NaN,NaN,NaN,NaN,112th Madison Avenue,New York,New York,New York,10029,July to November,Tue:8:00 am - 5:00 pm;Sat:8:00 am - 8:00 pm;,NaN,NaN,NaN,NaN,NaN,NaN,-73.949300,40.793900,Private business parking lot,N,N,Y,Y,N,-,Y,N,Y,Y,N,N,Y,Y,Y,Y,N,N,N,Y,N,N,Y,Y,N,N,N,N,N,N,N,N,N,N,N,3/1/2012 10:38:22 AM
5,1011100,12 South Farmers Market,http://www.12southfarmersmarket.com,12_South_Farmers_Market,@12southfrmsmkt,NaN,@12southfrmsmkt,3000 Granny White Pike,Nashville,Davidson,Tennessee,37204,05/05/2015 to 10/27/2015,Tue: 3:30 PM-6:30 PM;,NaN,NaN,NaN,NaN,NaN,NaN,-86.790709,36.118370,NaN,Y,N,N,N,Y,Y,Y,Y,N,Y,Y,N,Y,Y,Y,Y,Y,Y,N,N,N,Y,Y,Y,N,N,Y,N,Y,N,Y,Y,Y,N,N,5/1/2015 10:40:56 AM
6,1009845,125th Street Fresh Connect Farmers' Market,http://www.125thStreetFarmersMarket.com,https://www.facebook.com/125thStreetFarmersMarket,https://twitter.com/FarmMarket125th,NaN,Instagram--> 125thStreetFarmersMarket,"163 West 125th Street and Adam Clayton Powell,...",New York,New York,New York,10027,06/10/2014 to 11/25/2014,Tue: 10:00 AM-7:00 PM;,NaN,NaN,NaN,NaN,NaN,NaN,-73.948248,40.808953,Federal/State government building grounds,Y,Y,N,Y,Y,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,Y,Y,Y,N,Y,N,Y,Y,Y,N,Y,Y,N,Y,N,Y,N,N,N,N,4/7/2014 4:32:01 PM
7,1005586,12th & Brandywine Urban Farm Market,NaN,https://www.facebook.com/pages/12th-Brandywine...,NaN,NaN,https://www.facebook.com/delawareurbanfarmcoal...,12th & Brandywine Streets,Wilmington,New Castle,Delaware,19801,05/16/2014 to 10/17/2014,Fri: 8:00 AM-11:00 AM;,NaN,NaN,NaN,NaN,NaN,NaN,-75.534460,39.742117,"On a farm from: a barn, a greenhouse, a tent, ...",N,N,N,N,Y,N,N,N,N,N,N,N,Y,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,4/3/2014 3:43:31 PM
8,1008071,14&U Farmers' Market,NaN,https://www.facebook.com/14UFarmersMarket,https://twitter.com/14UFarmersMkt,NaN,NaN,1400 U Street NW,Washington,District of Columbia,District of Columbia,20009,05/03/2014 to 11/22/2014,Sat: 9:00 AM-1:00 PM;,NaN,NaN,NaN,NaN,NaN,NaN,-77.032050,38.916998,Other,Y,Y,Y,Y,Y,Y,Y,Y,N,Y,Y,N,Y,Y,Y,Y,N,Y,N,Y,Y,Y,N,N,N,N,N,Y,Y,Y,Y,N

In [4]:
#ensure dates are properly formatted
df['updateTime'] = pd.to_datetime(df['updateTime']) #does the heavy lifting
df['updateTime'] = df['updateTime'].apply(lambda x: x.strftime("%Y-%m-%d"))

In [5]:
#replace all nan values with empty strings for now
df = df.replace(np.nan, '', regex=True)

In [6]:
def form_address(rows):
    """
    Form address will return the address using the columns
    listed below, if any of the columns is not available
    it will return an empty string. 
    """
    cols = ['street', 'city', 'State', 'zip']
    if any([not rows[c] for c in cols]):
        return ''
    else:
        return ', '.join([rows[c] for c in cols])

#create standard address
address = df.apply(form_address, axis=1)
idx = df.columns.get_loc('zip') + 1
df.insert(idx, 'FullAddress', address)

In [7]:
#ensure bin columns are Y or N, if they are not Y defaults to N
bin_cols = df.iloc[:,23:-1].columns
for col in bin_cols:
    mask = mask = np.argwhere(df[col] != 'Y').ravel().tolist()
    df.loc[mask, col] = "N"

In [8]:
def is_valid_url(url):
    """
    Takes any string and returns it if it's a valid url
    returns and empty string otherwise.
    """
    regex = re.compile(
        r'^https?://'  # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+[A-Z]{2,6}\.?|'  # domain...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?'  # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    if regex.search(url):
        return url
    else:
        return ''

In [9]:
#validate website, facebook, and youtube
df['Facebook'] = df['Facebook'].apply(is_valid_url)
df['Website'] = df['Website'].apply(is_valid_url)
df['Youtube'] = df['Youtube'].apply(is_valid_url)

In [10]:
#ensure twitter uses @
def format_twitter(row):
    if 'https://twitter.com/' in row:
        return row.replace('https://twitter.com/', '@')
    elif row and '@' not in row:
        return '@' + row
    elif '@' in row:
        return row
    else:
        return ''

df['Twitter'] = df['Twitter'].apply(format_twitter)
#othermedia can stay as is, since it's a free form input

In [11]:
#drop x, y geo locations since a lot appear to be incorrect
df = df.drop(columns=['x','y'])

In [12]:
def is_valid_season(row):
    """
    Returns the row if the season in the format 
    "%m/%d/%Y" to "%m/%d/%Y", otherwise it sends the 
    row to another function for further inspecting.
    """
    pattern = re.compile("\d+/\d+/\d+ to \d+/\d+/\d+")
    if pattern.match(row.strip()):
        return row
    else:
        return get_valid_season(row)

In [13]:
def get_valid_season(row):
    """
    This is the second step for season dates inspection, 
    if the season is formatted with names of months it will use
    the current year for the season, otherwise it will send an 
    empty string back.
    """
    vals = row.strip().split(" to ")
    months = {v: k for k,v in enumerate(calendar.month_name)}
    del months[''] #empty not a valid month
    if not vals:
        return ''
    elif not all([val in list(months.keys()) for val in vals]):
        return ''
    elif len(vals) > 2: #we are only expecting two
        return ''
    else:
        year = datetime.datetime.now().year
        dt1 = datetime.datetime(year=year, month=months[vals[0]], day=1)
        dt2 = datetime.datetime(year=year, month=months[vals[1]], day=1)
        return dt1.strftime("%m/%d/%Y") + " to " + dt2.strftime("%m/%d/%Y")

In [14]:
#validate and correct all seasons dates
df['Season1Date'] = df['Season1Date'].apply(is_valid_season)
df['Season2Date'] = df['Season2Date'].apply(is_valid_season)
df['Season3Date'] = df['Season3Date'].apply(is_valid_season)
df['Season4Date'] = df['Season4Date'].apply(is_valid_season)

In [15]:
with pd.option_context('display.max_columns', None):
    display(df.head(10))

,FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,street,city,County,State,zip,FullAddress,Season1Date,Season1Time,Season2Date,Season2Time,Season3Date,Season3Time,Season4Date,Season4Time,Location,Credit,WIC,WICcash,SFMNP,SNAP,Organic,Bakedgoods,Cheese,Crafts,Flowers,Eggs,Seafood,Herbs,Vegetables,Honey,Jams,Maple,Meat,Nursery,Nuts,Plants,Poultry,Prepared,Soap,Trees,Wine,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime
0,1018261,Caledonia Farmers Market Association - Danville,https://sites.google.com/site/caledoniafarmers...,https://www.facebook.com/Danville.VT.Farmers.M...,,,,,Danville,Caledonia,Vermont,05828,,06/14/2017 to 08/30/2017,Wed: 9:00 AM-1:00 PM;,09/06/2017 to 10/18/2017,Wed: 2:00 PM-6:00 PM;,,,,,N,Y,Y,N,Y,N,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,Y,Y,Y,N,N,N,Y,Y,Y,Y,N,Y,Y,Y,N,N,Y,Y,N,N,2017-06-20
1,1018318,Stearns Homestead Farmers' Market,http://www.StearnsHomestead.com,,,,,6975 Ridge Road,Parma,Cuyahoga,Ohio,,,06/24/2017 to 09/30/2017,Sat: 9:00 AM-1:00 PM;,,,,,,,N,Y,N,N,Y,N,N,Y,N,Y,Y,Y,N,Y,Y,Y,Y,Y,N,N,N,N,Y,N,Y,N,N,N,N,Y,N,N,N,N,N,N,2017-06-21
2,1009364,106 S. Main Street Farmers Market,http://thetownofsixmile.wordpress.com/,,,,,106 S. Main Street,Six Mile,,South Carolina,29682,"106 S. Main Street, Six Mile, South Carolina, ...",,,,,,,,,N,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,2013-01-01
3,1010691,10th Steet Community Farmers Market,,,,,http://agrimissouri.com/mo-grown/grodetail.php...,10th Street and Poplar,Lamar,Barton,Missouri,64759,"10th Street and Poplar, Lamar , Missouri, 64759",04/02/2014 to 11/30/2014,Wed: 3:00 PM-6:00 PM;Sat: 8:00 AM-1:00 PM;,,,,,,,N,Y,N,N,N,N,N,Y,N,Y,N,Y,N,Y,Y,Y,Y,N,Y,N,N,Y,Y,Y,Y,N,N,N,N,Y,N,N,N,N,N,N,2014-10-28
4,1002454,112st Madison Avenue,,,,,,112th Madison Avenue,New York,New York,New York,10029,"112th Madison Avenue, New York, New York, 10029",07/01/2018 to 11/01/2018,Tue:8:00 am - 5:00 pm;Sat:8:00 am - 8:00 pm;,,,,,,,N,N,N,Y,Y,N,N,Y,N,Y,Y,N,N,Y,Y,Y,Y,N,N,N,Y,N,N,Y,Y,N,N,N,N,N,N,N,N,N,N,N,2012-03-01
5,1011100,12 South Farmers Market,http://www.12southfarmersmarket.com,,@12southfrmsmkt,,@12southfrmsmkt,3000 Granny White Pike,Nashville,Davidson,Tennessee,37204,"3000 Granny White Pike, Nashville, Tennessee, ...",05/05/2015 to 10/27/2015,Tue: 3:30 PM-6:30 PM;,,,,,,,N,Y,N,N,N,Y,Y,Y,Y,N,Y,Y,N,Y,Y,Y,Y,Y,Y,N,N,N,Y,Y,Y,N,N,Y,N,Y,N,Y,Y,Y,N,N,2015-05-01
6,1009845,125th Street Fresh Connect Farmers' Market,http://www.125thStreetFarmersMarket.com,https://www.facebook.com/125thStreetFarmersMarket,@FarmMarket125th,,Instagram--> 125thStreetFarmersMarket,"163 West 125th Street and Adam Clayton Powell,...",New York,New York,New York,10027,"163 West 125th Street and Adam Clayton Powell,...",06/10/2014 to 11/25/2014,Tue: 10:00 AM-7:00 PM;,,,,,,,N,Y,Y,N,Y,Y,Y,Y,Y,Y,Y,Y,N,Y,Y,Y,Y,Y,Y,N,Y,N,Y,Y,Y,N,Y,Y,N,Y,N,Y,N,N,N,N,2014-04-07
7,1005586,12th & Brandywine Urban Farm Market,,https://www.facebook.com/pages/12th-Brandywine...,,,https://www.facebook.com/delawareurbanfarmcoal...,12th & Brandywine Streets,Wilmington,New Castle,Delaware,19801,"12th & Brandywine Streets, Wilmington, Delawar...",05/16/2014 to 10/17/2014,Fri: 8:00 AM-11:00 AM;,,,,,,,N,N,N,N,N,Y,N,N,N,N,N,N,N,Y,Y,N,N,N,N,N,N,N,N,N,N,N,N,N,N,Y,N,N,N,N,N,N,2014-04-03
8,1008071,14&U Farmers' Market,,https://www.facebook.com/14UFarmersMarket,@14UFarmersMkt,,,1400 U Street NW,Washington,District of Columbia,District of Columbia,20009,"1400 U Street NW , Washington, District of Col...",05/03/2014 to 11/22/2014,Sat: 9:00 AM-1:00 PM;,,,,,,,N,Y,Y,Y,Y,Y,Y,Y,Y,N,Y,Y,N,Y,Y,Y,Y,N,Y,N,Y,Y,Y,N,N,N,N,N,Y,Y,Y,Y,N,N,N,N,2014-04-05
9,1012710,14th & Kennedy Street Farmers Market,,https://www.facebook.com/14KennnedyFarmersMarket/,@14KenFM,,instagram:14kenfm,"5500 Colorado Avenue, NW",Washington,District of Columbia,District of Columbia,20011,"5500 Colorado Avenue, NW, Washington , Distric...",04/09/2016 to 11/19/2016,Sat: 9:00 AM-1:00 PM;,,,,,,,N,Y,Y,Y,Y,Y,N,Y,N,Y,Y,Y,N,Y,Y,Y,Y,N,Y,N,N,N,Y,Y,N,N,N,Y,N,Y,N,N,N,Y,N,N

With pandas it is easy to create a database schema by using "create_engine" and save the farmers.db file for later use.

In [16]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///farmers.db')

In [17]:
df.to_sql('farmers', con=engine, if_exists='replace')

In [18]:
engine.execute("SELECT * FROM farmers").fetchall()

[(0, 1018261, ' Caledonia Farmers Market Association - Danville', 'https://sites.google.com/site/caledoniafarmersmarket/', 'https://www.facebook.com/Danville.VT.Farmers.Market/', '', '', '', '', 'Danville', 'Caledonia', 'Vermont', '05828', '', '06/14/2017 to 08/30/2017', 'Wed: 9:00 AM-1:00 PM;', '09/06/2017 to 10/18/2017', 'Wed: 2:00 PM-6:00 PM;', '', '', '', '', 'N', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'N', 'N', '2017-06-20'),
 (1, 1018318, " Stearns Homestead Farmers' Market", 'http://www.StearnsHomestead.com', '', '', '', '', '6975 Ridge Road', 'Parma ', 'Cuyahoga', 'Ohio', '', '', '06/24/2017 to 09/30/2017', 'Sat: 9:00 AM-1:00 PM;', '', '', '', '', '', '', 'N', 'Y', 'N', 'N', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N', 'N', 'N', 'Y', 'N', 'Y', 'N', 'N', 'N', 'N', 'Y', 'N', 'N', 'N', 'N', 'N', 'N', '2017-06-21'),
 (2, 10